# 문제
Jake 마켓의 마케팅 팀은 다가오는 명절 특수에 고객의 이목을 끌 새로운 이벤트를 기획했다.   
그 이름은 **Jake 럭키백**이다.   
   
럭키백은 구성품을 모른 채 먼저 구매하고, 배송받은 다음에야 비로소 구성품을 알 수 있는 상품이다.   
기간 한정으로 판매하는 럭키백의 물품은 생선으로 한정하기로 했다.   
하지만 럭키백 이벤트 소식을 들은 고객 만족 팀은 강하게 반대했다.   
   
어쩔 수 없이 마케팅 팀은 럭키백에 포함된 생선의 확률을 알려주는 방향으로 이벤트 수정안을 내놓았다.   
가령 A 럭키백에 도미 확률이 높다고 표시하면 도미를 원한 고객은 A 럭키백을 구매할 것이다.   
그렇다면 어떻게 생선의 확률을 구할지가 문제가 된다.   
머신러닝으로 럭키백의 생선이 어떤 타깃에 속하는지 확률을 구할 수 있을까?

# 럭키백의 확률
럭키백에 들어갈 수 있는 생선은 7개라고 해보자.   
이 이벤트를 잘 마치려면 럭키백에 들어간 생선의 크기, 무게 등이 주어졌을 때, 7개의 생선에 대한 확률을 출력해야 한다.   
이번에는 길이, 높이, 두께 외에도 대각선 길이와 무게도 사용할 수 있다.   
   
**k-최근접 이웃은 주변 이웃을 찾아주니까 이웃의 클래스 비율을 확률이라고 출력하면 되지 않을까?**   
   
가령 타깃 샘플 주위에 가장 가까운 이웃 샘플 10개가 있고 사각형이 3개, 삼각형이 5개, 원이 2개라고 해보자.   
이웃한 샘플의 클래스를 확률로 삼는다면 타깃 샘플이 사각형일 확률은 30%, 삼각형일 확률은 50%, 원일 확률은 20%이다.   
   
사이킷런의 k-최근접 이웃 분류기도 이와 동일한 방식으로 클래스 확률을 계산하여 제공한다.   
그럼 데이터를 준비하고 k-최근접 이웃 분류기로 럭키백에 들어간 생선의 확률을 계산해 본다.

# 데이터 준비하기
모델 훈련에 사용할 데이터를 만들어 본다.   
이번에도 판다스를 사용하여 인터넷에서 직접 CSV 데이터를 읽어 들인다.   
판다스의 `read_csv()` 함수로 CSV 파일을 데이터프레임으로 변환한 다음 `head()` 메서드로 처음 5개 행을 출력해 본다.

In [1]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


### 데이터프레임이란?
데이터프레임(dataframe)은 판다스에서 제공하는 2차원 표 형식의 주요 데이터 구조이다.   
데이터프레임은 넘파이 배열과 비슷하게 열과 행으로 이루여져 있다.   
데이터프레임은 통계와 그래프를 위한 메서드를 풍부하게 제공한다.   
또 데이터프레임은 넘파이로 상호 변환이 쉽고, 사이킷런과도 잘 호환된다.

그럼 어떤 종류의 생선이 있는지 Species 열에서 고유한 값을 추출해 본다.   
판다스의 `unique()` 함수를 사용하면 간단하다.

In [2]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


이 데이터프레임에서 Species 열을 타깃으로 만들고 나머지 5개 열은 입력 데이터로 사용한다.   
데이터프레임에서 열을 선택하는 방법은 간단하다.   
데이터프레임에서 원하는 열을 리스트로 나열하면 된다.   
Species 열을 빼고 나머지 5개 열을 선택해 본다.

In [3]:
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()

데이터프레임에서 여러 열을 선택하면 새로운 데이터프레임이 반환된다.   
이를 `to_numpy()` 메서드로 넘파이 배열로 바꾸어 fish_input에 저장했다.   
fish_input에 5개의 특성이 잘 저장되었는지 처음 5개 행을 출력해 본다.

In [4]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


입력 데이터가 잘 준비되었으므로 동일한 방식으로 타깃 데이터를 만든다.

In [5]:
fish_target = fish['Species'].to_numpy()

이제 데이터를 훈련 세트와 테스트 세트로 나눈다.

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42
)

그 다음 사이킷런의 `StandardScaler` 클래스를 사용해 훈련 세트와 테스트 세트를 표준화 전처리한다.

In [7]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

필요한 데이터를 모두 준비했다.   
이제 k-최근접 이웃 분류기로 테스트 세트에 들어 있는 확률을 예측해 본다.

# k-최근접 이웃 분류기의 확률 예측
사이킷런의 `KNeighborsClassifier` 클래스 객체를 만들고, 훈련 세트로 모델을 훈련한 다음 훈련 세트와 테스트 세트의 점수를 확인해 본다.   
최근접 이웃 개수인 k를 3으로 지정하여 사용한다.

In [8]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


현재 훈련 세트와 테스트 세트의 타깃 데이터에는 7개의 생선 종류가 들어가 있다.   
이렇게 타깃 데이터에 2개 이상의 클래스가 포함된 문제를 **다중 분류(multi-class classification)**라고 부른다.   
   
하지만 위의 코드에서 보듯이 이전까지 만들었던 이진 분류와 모델을 만들고 훈련하는 방식은 동일하다.   
이진 분류를 사용했을 때는 양성 클래스와 음성 클래스를 각각 1과 0으로 지정하여 타깃 데이터를 만들었다.   
다중 분류에서도 타깃값을 숫자로 바꾸어 입력할 수 있지만, 사이킷런에서는 편리하게도 문자열로 된 타깃값을 그대로 사용할 수 있다.   
   
이때 주의할 점은 타깃값을 그대로 사이킷런 모델에 전달하면 순서가 자동으로 알파벳 순으로 매겨진다.   
따라서 pd.unique(fish['Species'])로 출력했던 순서와 다르다.   
KNeighborsClassifier에서 정렬된 타깃값은 `classes_` 속성에 저장되어 있다.

In [9]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


`predict()` 메서드는 친절하게도 타깃값으로 예측을 출력한다.   
테스트 세트에 있는 처음 5개 샘플의 타깃값을 예측해 본다.

In [10]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


이 5개 샘플에 대한 예측은 어떤 확률로 만들어졌을까?   
사이킷런의 분류 모델은 `predict_proba()` 메서드로 클래스별 확률값을 반환한다.   
테스트 세트에 있는 처음 5개의 샘플에 대한 확률을 출력해 본다.   
넘파이 round() 함수는 기본으로 소수점 첫째 자리에서 반올림을 하는데, decimals 매개변수로 유지할 소수점 아래 자릿수를 지정할 수 있다.

In [11]:
import numpy as np

proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


predict_proba() 메서드의 출력 순서는 앞서 보았던 classes_ 속성과 같다.   
즉 첫 번째 열이 'Bream'에 대한 확률, 두 번째 열이 'Parkki'에 대한 확률이다.   
   
이 모델이 계산한 확률이 가장 가까운 이웃의 비율과 맞는지 확인해 본다.   
네 번째 샘플의 최근접 이웃의 클래스를 확인해 본다.

In [12]:
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

[['Roach' 'Perch' 'Perch']]


이 샘플의 이웃은 다섯 번째 클래스인 'Roach'가 1개이고, 세 번째 클래스인 'Perch'가 2개이다.   
따라서 다섯 번째 클래스에 대한 확률은 **1/3 = 0.3333**이고, 세 번째 클래스에 대한 확률은 **2/3 = 0.6667**이 된다.   
앞서 출력한 네 번째 샘플의 클래스 확률과 같다.   
   
성공인 것 같으나 뭔가 좀 이상하다.   
3개의 최근접 이웃을 사용하기 때문에 가능한 확률은 **0/3, 1/3, 2/3, 3/3**이 전부이다.   
만약 럭키백의 확률을 이렇게만 표시한다면 마케팅 팀이 만족하지 않을 것 같다.   
확률이라고 말하기 어색하기도 하다.   
좀 더 좋은 방법을 모색해야 한다.

# 로지스틱 회귀
**로지스틱 회귀(logistic regression)**는 이름은 회귀이지만 분류 모델이다.   
이 알고리즘은 선형 회귀와 동일하게 선형 방정식을 학습한다.   
예를 들면 다음과 같다.

$$z = a*(Weight) + b*(Length) + c*(Diagonal) + d*(Height) + e*(Width) + f$$

여기에서 a, b, c, d, e는 가중치 혹은 계수이다.   
특성은 늘어났지만 앞서 다룬 다중 회귀를 위한 선형 방정식과 동일하다.   
z는 어떤 값도 가능하다.   
**하지만 확률이 되려면 0 ~ 1(또는 0 ~ 100%) 사이 값이 되어야 한다.**   
z가 아주 큰 움수일 때 0이 되고, z가 아주 큰 양수일 때 1이 되도록 바꾸는 방법은 없을까?   
**시그모이드 함수(sigmoid function)**를 사용하면 가능하다.

$$f(z) = \frac{1} {1 + e^{-z}}$$

위쪽의 식이 시그모이드 함수이다.   
선형 방정식의 출력 z의 음수를 사용해 자연 상수 e를 거듭제곱 하고 1을 더한 값의 역수를 취한다.   
이렇게 복잡하게 계산한 이유는 아래와 같은 그래프를 만들 수 있기 때문이다.

![sigmoid](https://github.com/kyomin/algorithm/assets/46395776/a10b6752-1a6a-467d-ba6c-8a022d63a6e6)

z가 무한하게 큰 음수일 경우 이 함수는 0에 가까워지고,   
z가 무안하게 큰 양수가 될 때는 1에 가까워진다.   
z가 0일 때는 0.5가 된다.   
즉, z가 어떤 값이 되더라도 f(z)는 절대로 0 ~ 1 사이의 범위를 벗어날 수 없다.   
그렇다면 0 ~ 1 사이의 값을 0 ~ 100%까지 확률로 해석할 수 있게 된다.

이 원리를 이용해 로지스틱 회귀 모델을 훈련해 본다.   
당연히 사이킷런에는 로지스틱 회귀 모델인 `LogisticRegression` 클래스가 준비되어 있다.   
   
훈련하기 전에 간단한 이진 분류를 수행해 본다.   
**이진 분류일 경우 시그모이드 함수의 출력이 0.5보다 크면 양성 클래스, 0.5보다 작으면 음성 클래스로 판단한다.**   
그럼 먼저 도미와 빙어 2개를 사용해서 이진 분류를 수행해 본다.

# 로지스틱 회귀로 이진 분류 수행하기
넘파이 배열은 True, False 값을 전달하여 행을 선택할 수 있다.   
이를 **불리언 인덱싱(boolean indexing)**이라고 한다.   
   
간단한 예를 보면 금방 이해할 수 있다.   
다음과 같이 'A'에서 'E'까지 5개의 원소로 이루어진 배열이 있다.   
여기서 'A'와 'C'만 골라내려면 첫 번째와 세 번째 원소만 True이고 나머지 원소는 모두 False인 배열을 전달하면 된다.

In [13]:
char_arr = np.array(['A', 'B', 'C', 'D', 'E'])
print(char_arr[[True, False, True, False, False]])

['A' 'C']


이와 같은 방식을 사용해 훈련 세트에서 도미(Bream)와 빙어(Smelt)의 행만 골라낸다.   
비교 연산자를 사용하면 도미와 빙어의 행을 모두 True로 만들 수 있다.   
예를 들어 도미인 행을 골라내려면 train_target == 'Bream'과 같이 쓴다.   
이 비교식은 train_target 배열에서 'Bream'인 것은 True이고, 그 외는 모두 False인 배열을 반환한다.   
도미와 빙어에 대한 비교 결과를 비트 OR 연산자(|)를 사용해 합치면 도미와 빙어에 대한 행만 골래날 수 있다.

In [14]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

bream_smelt_indexes 배열은 도미와 빙어일 경우 True이고 그 외는 모두 False 값이 들어가 있다.   
따라서 이 배열을 사용해 train_scaled와 train_target 배열에 불리언 인덱싱을 적용하면 손쉽게 도미와 빙어 데이터만 골라낼 수 있다.   
   
이제 이 데이터로 로지스틱 회귀 모델을 훈련해 본다.   
`LogisticRegression` 클래스는 선형 모델이므로 sklearn.linear_model 패키지 아래 있다.

In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

훈련한 모델을 사용해 train_bream_smelt에 있는 처음 5개 샘플을 예측해 본다.

In [16]:
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


두 번째 샘플을 제외하고는 모두 도미로 예측했다.   
KNeighborsClassifier와 마찬가지로 예측 확률은 predict_proba() 메서드에서 제공한다.   
train_bream_smelt에서 처음 5개 샘플의 예측 확률을 출력해 본다.

In [17]:
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


샘플마다 2개의 확률이 출력되었다.   
첫 번째 열이 음성 클래스(0)에 대한 확률이고, 두 번째 열이 양성 클래스(1)에 대한 확률이다.   
그럼 Bream과 Smelt 중에 어떤 것이 양성 클래스일까?   
앞서 k-최근접 이웃 분류기에서 보았듯이 사이킷런은 타깃값을 알파벳순으로 정렬하여 사용한다.   
이를 classes_ 속성에서 확인해 본다.

In [18]:
print(lr.classes_)

['Bream' 'Smelt']


빙어(Smelt)가 양성 클래스이다.   
`predict_proba()` 메서드가 반환한 배열 값을 보면 두 번째 샘플만 양성 클래스인 빙어의 확률이 높다.   
나머지는 모두 도미(Bream)로 예측할 것이다.   
   
그러면 선형 회귀에서처럼 로지스틱 회귀가 학습한 계수를 확인해 본다.

In [19]:
print(lr.coef_, lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


따라서 이 로지스틱 회귀 모델이 학습한 방정식은 다음과 같다.

$$z = -0.404*(Weight) - 0.576*(Length) - 0.663*(Diagonal) - 1.013*(Height) - 0.732*(Width) - 2.161$$

확실히 로지스틱 회귀는 선형 회귀와 매우 비슷하다.   
그럼 `LogisticRegression` 모델로 z값을 계산해 볼 수 있을까?   
가능하다.   
LogisticRegression 클래스는 `decision_function()` 메서드로 z 값을 출력할 수 있다.   
train_bream_smelt의 처음 5개 샘플의 z 값을 출력해 본다.

In [20]:
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


이 z 값을 시그모이드 함수에 통과시키면 확률을 얻을 수 있다.   
다행히 파이썬의 사이파이(scipy) 라이브러리에도 시그모이드 함수가 있다.   
바로 expit()이다.   
np.exp() 함수를 사용해 분수 계산을 하는 것보다 훨씬 편리하고 안전하다.   
decisions 배열의 값을 확률로 변환해 본다.

In [21]:
from scipy.special import expit
print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


출력된 값을 보면 predict_proba() 메서드 출력의 두 번째 열의 값과 동일하다.   
즉 decision_function() 메서드는 양성 클래스에 대한 z 값을 반환한다.   
   
이진 분류를 위해 2개의 생선 샘플을 골라냈고 이를 사용해 로지스틱 회귀 모델을 훈련했다.   
이진 분류일 경우 predict_proba() 메서드는 음성 클래스와 양성 클래스에 대한 확률을 출력한다.   
또 decision_function() 메서드는 양성 클래스에 대한 z 값을 계산한다.   
또 coef_ 속성과 intercept_ 속성에는 로지스틱 모델이 학습한 선형 방정식의 계수가 들어 있다.   
   
이제 이진 분류의 경험을 바탕으로 7개의 생선을 분류하는 다중 분류 문제로 넘어간다.

# 로지스틱 회귀로 다중 분류 수행하기
앞서 이진 분류를 위해 로지스틱 회귀 모델을 훈련시켜 보았다.   
다중 분류도 크게 다르지 않다.   
여기에서도 `LogisticRegression` 클래스를 사용해 7개의 생선을 분류해 보면서 이진 분류와의 차이점을 알아본다.   
   
LogisticRegression 클래스는 기본적으로 반복적인 알고리즘을 사용한다.   
max_iter 매개변수에서 반복 횟수를 지정하며 기본값은 100이다.   
여기에 준비한 데이터셋을 사용해 모델을 훈련하면 반복 횟수가 부족하다는 경고가 발생한다.   
충분하게 훈련시키기 위해 반복 횟수를 1,000으로 늘린다.   
   
또 LogisticRegression은 기본적으로 릿지 회귀와 같이 계수의 제곱을 규제한다.   
이런 규제를 L2 규제라고도 부른다.   
릿지 회귀에서는 alpha 매개변수로 규제의 양을 조절했다.   
alpha가 커지면 규제도 커진다.   
LogisticRegression에서 규제를 제어하는 매개변수는 C이다.   
하지만 C는 alpha와 반대로 값이 작을수록 규제가 커진다.   
C의 기본값은 1이다.   
여기에서는 규제를 조금 완화하기 위해 20으로 늘린다.   
   
다음 코드는 LogisticRegression 클래스로 다중 분류 모델을 훈련하는 코드이다.   
7개의 생선 데이터가 모두 들어 있는 train_scaled와 train_target을 사용한 점을 눈여겨 본다.

In [22]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


훈련 세트와 테스트 세트에 대한 점수가 높고 과대적합이나 과소적합으로 치우친 것 같지 않다.   
다음으로 테스트 세트의 처음 5개 샘플에 대한 예측을 출력해 본다.

In [23]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


이번에는 테스트 세트의 처음 5개 샘플에 대한 예측 확률을 출력해 본다.   
출력을 간소하게 하기 위해 소수점 네 번째 자리에서 반올림한다.

In [24]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


5개 샘플에 대한 예측이므로 5개의 행이 출력되었다.   
또 7개 생선에 대한 확률을 계산했으므로 7개의 열이 출력되었다.   
이진 분류일 경우 2개의 열만 있었다는 것을 기억한다.   
   
첫 번째 샘플을 보면 세 번째 열의 확률이 가장 높다.   
백분율 변환 시 84.1%나 된다.   
그렇다면 세 번째 열이 농어(Perch)에 대한 확률일까?   
classes_ 속성에서 클래스 정보를 확인해 본다.

In [25]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


첫 번째 샘플은 Perch를 가장 높은 확률로 예측했다.   
두 번째 샘플은 여섯 번째 열인 Smelt를 가장 높은 확률(94.6%)로 예측했다.   
   
다중 분류도 어렵지 않다.   
이진 분류는 샘플마다 2개의 확률을 출력하고 다중 분류는 샘플마다 클래스 개수만큼 확률을 출력한다.   
여기에서는 7개이다.   
이 중에서 가장 높은 확률이 예측 클래스가 된다.   
   
그럼 다중 분류일 경우 선형 방정식은 어떤 모습일까?   
coef_와 intercept_의 크기를 출력해 본다.

In [26]:
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


이 데이터는 5개의 특성을 사용하므로 coef_ 배열의 열은 5개이다.   
그런데 행이 7이다.   
intercept_도 7개나 있다.   
이 말은 이진 분류에서 보았던 z를 7개나 계산한다는 의미이다.   
즉, **다중 분류는 클래스마다 z 값을 하나씩 계산한다.**   
당연히 가장 높은 z 값을 출력하는 클래스가 예측 클래스가 된다.   
그럼 확률은 어떻게 계산한 것일까?   
이진 분류에서는 시그모이드 함수를 사용해 z를 0과 1 사이의 값으로 변환했다.   
다중 분류는 이와 달리 **소프트맥스(softmax)** 함수를 사용하여 7개의 z 값을 확률로 변환한다.

### 소프트맥스 함수란?
시그모이드 함수는 하나의 선형 방정식의 출력값을 0 ~ 1 사이로 압축한다.   
이와 달리 소프트맥스 함수는 여러 개의 선형 방정식의 출력값을 0 ~ 1 사이로 압축하고 전체 합이 1이 되도록 만든다.   
이를 위해 지수 함수를 사용하기 때문에 **정규화된 지수 함수**라고도 부른다.

소프트맥스도 어렵지 않다.   
차근차근 계산 방식을 짚어 본다.   
먼저 7개의 z 값의 이름을 z1에서 z7으로 붙인다.   
z1 ~ z7까지 값을 사용해 지수 함수 e^z1 ~ e^z7을 계산해 모두 더한다.   
이를 eSum이라고 한다.

$$eSum = e^{z1} + e^{z2} + e^{z3} + e^{z4} + e^{z5} + e^{z6} + e^{z7}$$

그 다음 e^z1 ~ e^z7을 각각 eSum으로 나누어 주면 된다.

$$s1 = \frac{e^{z1}} {eSum}, s2 = \frac{e^{z2}} {eSum}, ..., s7 = \frac{e^{z7}} {eSum}$$

s1에서 s7까지 모두 더하면 분자와 분모가 같아지므로 1이 된다.   
7개 생선에 대한 확률의 합은 1이 되어야 하므로 잘 맞다.

그럼 이진 분류에서처럼 decision_function() 메서드로 z1 ~ z7까지의 값을 구한 다음 소프트맥스 함수를 사용해 확률로 바꾸어 본다.   
먼저 테스트 세트의 처음 5개 샘플에 대한 z1 ~ z7의 값을 구해 본다.

In [27]:
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


역시 사이파이는 소프트맥스 함수도 제공한다.   
scipy.special 아래에 softmax() 함수를 임포트해 사용한다.

In [28]:
from scipy.special import softmax
proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


앞서 구한 decision 배열을 softmax() 함수에 전달했다.   
softmax()의 axis 매개변수는 소프트맥스를 계산할 축을 지정한다.   
여기에서는 axis=1로 지정하여 각 행, 즉 각 샘플에 대해 소프트맥스를 계산한다.   
만약 axis 매개변수를 지정하지 않으면 배열 전체에 대해 소프트맥스를 계산한다.   
   
출력 결과를 앞서 구한 proba 배열과 비교해 본다.   
결과가 정확히 일치한다!